How to move from legacy LangChain agents to more flexible LangGraph agents. LangChain agents (the AgentExecutor in particular) have multiple configuration parameters.

In [4]:
from dotenv import load_dotenv, dotenv_values
import google.generativeai as genai
from IPython.display import Markdown, display
import os 


load_dotenv()
os.getenv("GOOGLE_API_KEY") 
my_api_key = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=my_api_key)

In [6]:
from langchain_google_genai.chat_models import  ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(model= "gemini-1.5-flash")

In [9]:
from langchain_core.tools import tool

@tool
def magic_function(input: int) -> int:
    """Applies a magic function to an input."""
    return input + 2


tools = [magic_function]


query = "what is the value of magic_function(3)?"

In [10]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("human", "{input}"),
        # Placeholders fill up a **list** of messages
        ("placeholder", "{agent_scratchpad}"),
    ]
)


agent = create_tool_calling_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

agent_executor.invoke({"input": query})

{'input': 'what is the value of magic_function(3)?',
 'output': 'The value of magic_function(3) is: `{"output": 5}`. \n'}

LangGraph's react agent executor manages a state that is defined by a list of messages. It will continue to process the list until there are no tool calls in the agent's output. To kick it off, we input a list of messages. The output will contain the entire state of the graph-- in this case, the conversation history.



In [11]:
from langgraph.prebuilt import create_react_agent

app = create_react_agent(model, tools)


messages = app.invoke({"messages": [("human", query)]})
{
    "input": query,
    "output": messages["messages"][-1].content,
}

{'input': 'what is the value of magic_function(3)?',
 'output': 'The value of magic_function(3) is  `{"output": 5}`. \n'}

In [12]:
message_history = messages["messages"]

new_query = "Pardon?"

messages = app.invoke({"messages": message_history + [("human", new_query)]})
{
    "input": new_query,
    "output": messages["messages"][-1].content,
}

{'input': 'Pardon?',
 'output': 'The `magic_function` applied to the input `3` returns a dictionary with a single key-value pair: `"output": 5`. \n\nIn other words, the function transforms the input `3` into the output value `5`. \n'}

##### Prompt Templates
With legacy LangChain agents you have to pass in a prompt template. You can use this to control the agent.

With LangGraph react agent executor, by default there is no prompt. You can achieve similar control over the agent in a few ways:

    Pass in a system message as input
    Initialize the agent with a system message
    Initialize the agent with a function to transform messages before passing to the model.

LangGraph's prebuilt create_react_agent does not take a prompt template directly as a parameter, but instead takes a messages_modifier parameter. This modifies messages before they are passed into the model, and can be one of four values:

    A SystemMessage, which is added to the beginning of the list of messages.
    A string, which is converted to a SystemMessage and added to the beginning of the list of messages.
    A Callable, which should take in a list of messages. The output is then passed to the language model.
    Or a Runnable, which should should take in a list of messages. The output is then passed to the language model.

In [23]:
from langchain_core.messages import SystemMessage
from langgraph.prebuilt import create_react_agent

system_message = "You are a helpful assistant. Respond only in English."
# This could also be a SystemMessage object
# system_message = SystemMessage(content="You are a helpful assistant. Respond only in Spanish.")

app = create_react_agent(model, tools, messages_modifier=system_message)
messages = app.invoke({"messages": [("user", query)]})

We can also pass in an arbitrary function. This function should take in a list of messages and output a list of messages. We can do all types of arbitrary formatting of messages here. In this cases, let's just add a SystemMessage to the start of the list of messages.

In [28]:
from langchain_core.messages import AnyMessage
from langgraph.prebuilt import create_react_agent

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Respond only in French."),
        ("placeholder", "{messages}"),
    ]
)


def _modify_messages(messages: list[AnyMessage]):
    return prompt.invoke({"messages": messages}).to_messages() + [
        ("user", "Also say 'Bravo!' after the answer.")
    ]


app = create_react_agent(model, tools, messages_modifier=_modify_messages)


messages = app.invoke({"messages": [("human", query)]})
print(
    {
        "input": query,
        "output": messages["messages"][-1].content,
    }
)

{'input': 'what is the value of magic_function(3)?', 'output': 'La valeur de magic_function(3) est  "{"output": 5}" . Bravo! \n'}


##### Memory
    ###### In LangChain: 
    With LangChain's AgentExecutor, you could add chat Memory so it can engage in a multi-turn conversation.

    ###### In LangGraph
    Memory is just persistence, aka checkpointing.   
    Add a checkpointer to the agent and you get chat memory for free.



In [29]:
from langchain_core.messages import SystemMessage
from langgraph.checkpoint import MemorySaver  # an in-memory checkpointer
from langgraph.prebuilt import create_react_agent

system_message = "You are a helpful assistant."
# This could also be a SystemMessage object
# system_message = SystemMessage(content="You are a helpful assistant. Respond only in Spanish.")

memory = MemorySaver()
app = create_react_agent(
    model, tools, messages_modifier=system_message, checkpointer=memory
)

config = {"configurable": {"thread_id": "test-thread"}}
print(
    app.invoke(
        {
            "messages": [
                ("user", "Hi, I'm polly! What's the output of magic_function of 3?")
            ]
        },
        config,
    )["messages"][-1].content
)
print("---")
print(
    app.invoke({"messages": [("user", "Remember my name?")]}, config)["messages"][
        -1
    ].content
)
print("---")
print(
    app.invoke({"messages": [("user", "what was that output again?")]}, config)[
        "messages"
    ][-1].content
)

The output of magic_function(3) is:  {"output": 5} 

---
Yes, I remember your name is Polly! 😊 

---
The output of `magic_function(3)` was  `{"output": 5}`. 

Is there anything else I can help you with? 



##### Iterating through steps      

With LangChain's AgentExecutor, you could iterate over the steps using the stream (or async astream) methods or the iter method. LangGraph supports stepwise iteration using stream        

In LangGraph, things are handled natively using stream or the asynchronous astream method.        |

In [35]:
from langchain_core.messages import AnyMessage
from langgraph.prebuilt import create_react_agent

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("placeholder", "{messages}"),
    ]
)


def _modify_messages(messages: list[AnyMessage]):
    return prompt.invoke({"messages": messages}).to_messages()


app = create_react_agent(model, tools, messages_modifier=_modify_messages)


for step in app.stream({"messages": [("human", query)]}, stream_mode="updates"):
    print(step)

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'magic_function', 'arguments': '{"input": 3.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-a3c90310-6f82-463e-a204-dfcbc4c8062a-0', tool_calls=[{'name': 'magic_function', 'args': {'input': 3.0}, 'id': '28fca33c-8eeb-42fb-b845-32468e200d79'}], usage_metadata={'input_tokens': 58, 'output_tokens': 15, 'total_tokens': 73})]}}
{'tools': {'messages': [ToolMessage(content='5', name='magic_function', tool_call_id='28fca33c-8eeb-42fb-b845-32468e200

##### return_intermediate_steps
In LangChain, setting this parameter on AgentExecutor allows users to access intermediate_steps, which pairs agent actions (e.g., tool invocations) with their outcomes.

In LangGraph, By default the react agent executor in LangGraph appends all messages to the central state. Therefore, it is easy to see any intermediate steps by just looking at the full state.

In [36]:
from langgraph.prebuilt import create_react_agent

app = create_react_agent(model, tools=tools)

messages = app.invoke({"messages": [("human", query)]})

messages

{'messages': [HumanMessage(content='what is the value of magic_function(3)?', id='c8c3772b-14e1-4176-b610-f9650fbd3451'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'magic_function', 'arguments': '{"input": 3.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-c719edd4-1af2-4689-b810-c9096150ff1b-0', tool_calls=[{'name': 'magic_function', 'args': {'input': 3.0}, 'id': '070c72d6-b7e7-453d-be21-d28f29d94af1'}], usage_metadata={'input_tokens': 52, 'output_tokens': 15, 'total_tokens': 67}),
  ToolMessage(content

##### max_iterations
In LangChain, AgentExecutor implements a max_iterations parameter, allowing users to abort a run that exceeds a specified number of iterations.

In LangGraph, this is controlled via recursion_limit configuration parameter.Note that in AgentExecutor, an "iteration" includes a full turn of tool invocation and execution. In LangGraph, each step contributes to the recursion limit, so we will need to multiply by two (and add one) to get equivalent results.

If the recursion limit is reached, LangGraph raises a specific exception type, that we can catch and manage similarly to AgentExecutor.

In [37]:
from langgraph.errors import GraphRecursionError
from langgraph.prebuilt import create_react_agent

RECURSION_LIMIT = 2 * 3 + 1

app = create_react_agent(model, tools=tools)

try:
    for chunk in app.stream(
        {"messages": [("human", query)]},
        {"recursion_limit": RECURSION_LIMIT},
        stream_mode="values",
    ):
        print(chunk["messages"][-1])
except GraphRecursionError:
    print({"input": query, "output": "Agent stopped due to max iterations."})

content='what is the value of magic_function(3)?' id='e5ba1072-6774-4c57-a8cc-ea63ed1574dc'
content='' additional_kwargs={'function_call': {'name': 'magic_function', 'arguments': '{"input": 3.0}'}} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]} id='run-36811a76-5b92-4998-ab88-72e283029d5e-0' tool_calls=[{'name': 'magic_function', 'args': {'input': 3.0}, 'id': '5a649215-e4a6-444e-98f4-534f216f9add'}] usage_metadata={'input_tokens': 52, 'output_tokens': 15, 'total_tokens': 67}
content='5' name='magic_function' id='895034e9-8ff8-4572-8013-8aaca02d

##### max_execution_time
In LangChain, AgentExecutor implements a max_execution_time parameter, allowing users to abort a run that exceeds a total time limit.

In LangGraph
With LangGraph's react agent, you can control timeouts on two levels.

You can set a step_timeout to bound each step:

In [38]:
from langgraph.prebuilt import create_react_agent

app = create_react_agent(model, tools=tools)
# Set the max timeout for each step here
app.step_timeout = 2

try:
    for chunk in app.stream({"messages": [("human", query)]}):
        print(chunk)
        print("------")
except TimeoutError:
    print({"input": query, "output": "Agent stopped due to max iterations."})

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'magic_function', 'arguments': '{"input": 3.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-da7414e4-50fe-49ed-8efa-835aba257012-0', tool_calls=[{'name': 'magic_function', 'args': {'input': 3.0}, 'id': '84aa6bd2-39f1-4097-abc5-b632555950ed'}], usage_metadata={'input_tokens': 52, 'output_tokens': 15, 'total_tokens': 67})]}}
------
{'tools': {'messages': [ToolMessage(content='5', name='magic_function', tool_call_id='84aa6bd2-39f1-4097-abc5-b6

The other way to set a single max timeout for an entire run is to directly use the python stdlib asyncio library.

In [39]:
import asyncio

from langgraph.prebuilt import create_react_agent

app = create_react_agent(model, tools=tools)


async def stream(app, inputs):
    async for chunk in app.astream({"messages": [("human", query)]}):
        print(chunk)
        print("------")


try:
    task = asyncio.create_task(stream(app, {"messages": [("human", query)]}))
    await asyncio.wait_for(task, timeout=3)
except TimeoutError:
    print("Task Cancelled.")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'magic_function', 'arguments': '{"input": 3.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-8add88cc-b1a4-44df-89b6-4386eb5dc4be-0', tool_calls=[{'name': 'magic_function', 'args': {'input': 3.0}, 'id': 'deb78b20-2d77-4e4b-b61b-6b081cd6c155'}], usage_metadata={'input_tokens': 52, 'output_tokens': 15, 'total_tokens': 67})]}}
------
{'tools': {'messages': [ToolMessage(content='5', name='magic_function', tool_call_id='deb78b20-2d77-4e4b-b61b-6b

##### Early_stopping_method
In LangChain
With LangChain's AgentExecutor, you could configure an early_stopping_method to either return a string saying "Agent stopped due to iteration limit or time limit." ("force") or prompt the LLM a final time to respond ("generate").

In LangGraph, you can explicitly handle the response behavior outside the agent, since the full state can be accessed.



In [40]:
from langgraph.errors import GraphRecursionError
from langgraph.prebuilt import create_react_agent

RECURSION_LIMIT = 2 * 1 + 1

app = create_react_agent(model, tools=tools)

try:
    for chunk in app.stream(
        {"messages": [("human", query)]},
        {"recursion_limit": RECURSION_LIMIT},
        stream_mode="values",
    ):
        print(chunk["messages"][-1])
except GraphRecursionError:
    print({"input": query, "output": "Agent stopped due to max iterations."})

content='what is the value of magic_function(3)?' id='19ce4b7a-e7f6-4f81-b5f1-36b2a2241c29'
content='' additional_kwargs={'function_call': {'name': 'magic_function', 'arguments': '{"input": 3.0}'}} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]} id='run-fe791725-f9a5-4933-88ff-dc1cf2478163-0' tool_calls=[{'name': 'magic_function', 'args': {'input': 3.0}, 'id': '521a39ef-3c12-4ed3-a6ba-592e965c7dd2'}] usage_metadata={'input_tokens': 52, 'output_tokens': 15, 'total_tokens': 67}
content='5' name='magic_function' id='e8d353c8-d962-4776-b358-6d9a2358

##### trim_intermediate_steps
With LangChain's AgentExecutor, you could trim the intermediate steps of long-running agents using trim_intermediate_steps, which is either an integer (indicating the agent should keep the last N steps) or a custom function.

In LangGraph, We can use the messages_modifier just as before when passing in prompt templates.


In [41]:
from langchain_core.messages import AnyMessage
from langgraph.errors import GraphRecursionError
from langgraph.prebuilt import create_react_agent

magic_step_num = 1


@tool
def magic_function(input: int) -> int:
    """Applies a magic function to an input."""
    global magic_step_num
    print(f"Call number: {magic_step_num}")
    magic_step_num += 1
    return input + magic_step_num


tools = [magic_function]


def _modify_messages(messages: list[AnyMessage]):
    # Give the agent amnesia, only keeping the original user query
    return [("system", "You are a helpful assistant"), messages[0]]


app = create_react_agent(model, tools, messages_modifier=_modify_messages)

try:
    for step in app.stream({"messages": [("human", query)]}, stream_mode="updates"):
        pass
except GraphRecursionError as e:
    print("Stopping agent prematurely due to triggering stop condition")

Call number: 1
Call number: 2
Call number: 3
Call number: 4
Call number: 5
Call number: 6
Call number: 7
Call number: 8
Call number: 9
Call number: 10
Call number: 11
Call number: 12
Stopping agent prematurely due to triggering stop condition
